# **Homework 2**

## Setup ##

In [1]:
import gdown
import zipfile
import os

In [2]:
#Check if files exists in data folder
if os.path.exists('data/'):
    print('Files already downloaded.')

else:
    output_path = 'data.zip'
    file_id = '1KDN-rFCq9IDJ7_kNW5y5Co100KNpklz-'
    url = f'https://drive.google.com/uc?id={file_id}'
    # Download the zip file
    gdown.download(url, output_path, quiet=False)

    # Extract the contents of the zip file
    with zipfile.ZipFile(output_path, 'r') as zip_ref:
        zip_ref.extractall('data')

    # Remove the zip file
    os.remove(output_path)


Files already downloaded.


In [3]:
if(os.path.exists('test/') and os.path.exists('train')):
    print('Files already extracted')
else:
    print('Extracting the test.zip and train.zip files...')
    # Extract the test.zip file
    with zipfile.ZipFile('data/public/test.zip', 'r') as zip_ref:
        zip_ref.extractall()

    # Extract the train.zip file
    with zipfile.ZipFile('data/public/train.zip', 'r') as zip_ref:
        zip_ref.extractall()

    print('Done!')

Files already extracted


## First Iteration ##

In [4]:
import tensorflow as tf

# Check if GPU is available
print('GPU is', 'available' if tf.config.list_physical_devices('GPU') else 'NOT AVAILABLE')

# Enable GPU
physical_devices = tf.config.list_physical_devices('GPU')

GPU is available


In [1]:
from keras_preprocessing.image import ImageDataGenerator

# Define the path to your training data
trainingset = 'train/'
validationset = 'test/'

batch_size = 64

# Define batch size and input shape
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
)

# Augment training data
train_generator = train_datagen.flow_from_directory(
    directory=trainingset,
    target_size=(96, 96),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    subset='training'
)

# No augmentation for validation data
validation_datagen = ImageDataGenerator(rescale=1. / 255)

validation_generator = validation_datagen.flow_from_directory(
    directory=validationset,
    target_size=(96, 96),
    batch_size=batch_size,
    shuffle=False,
    class_mode='categorical'
)  # set as validation data

num_samples = train_generator.n
num_classes = train_generator.num_classes
input_shape = train_generator.image_shape

classnames = [k for k, v in train_generator.class_indices.items()]
img_h = input_shape[0]
img_w = input_shape[1]
print("Image height = %d, Image Width = %d" % (img_h, img_w))
print("Image input %s" % str(input_shape))
print("Classes: %r" % classnames)
print('Loaded %d training samples from  %d classes.' % (num_samples, num_classes))
print('Loaded %d test samples from %d classes.' % (validation_generator.n, validation_generator.num_classes))


Found 6369 images belonging to 5 classes.
Found 2749 images belonging to 5 classes.
Image height = 96, Image Width = 96
Image input (96, 96, 3)
Classes: ['0', '1', '2', '3', '4']
Loaded 6369 training samples from  5 classes.
Loaded 2749 test samples from 5 classes.


In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten,\
                         Conv2D, MaxPooling2D
from keras import optimizers

def MyCNN(input_shape, num_classes):
    model = Sequential(name="MyOptimizedCNN")

    # C1 Convolutional Layer 
    model.add(Conv2D(filters=15, input_shape=input_shape, kernel_size=(4,4)))
    model.add(Activation('relu'))

    # C2 Convolutional Layer
    model.add(Conv2D(filters=20, kernel_size=(4,4)))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2)))

    # C3 Convolutional Layer
    model.add(Conv2D(filters=30, kernel_size=(4,4)))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2)))

    # Flatten
    model.add(Flatten())

    # D1 Dense Layer
    model.add(Dense(128))
    model.add(Activation('relu'))
    # Dropout
    model.add(Dropout(0.4))
    # D2 Dense Layer
    model.add(Dense(96))
    model.add(Activation('relu'))
    # Dropout
    model.add(Dropout(0.4))

    # Output Layer
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # Compile

    optimizer = optimizers.RMSprop(lr=0.0001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# create the model
# Input shape is (3, 96, 96) for the RGB image
model = MyCNN(input_shape, num_classes)
model.summary()

Model: "MyOptimizedCNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 93, 93, 15)        735       
                                                                 
 activation (Activation)     (None, 93, 93, 15)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 20)        4820      
                                                                 
 activation_1 (Activation)   (None, 90, 90, 20)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 45, 45, 20)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 42, 42, 30)        9630      
                                                    

c:\Users\andri\miniconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [72]:
from keras import callbacks

# Define callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Calculate steps per epoch and validation steps
steps_per_epoch = len(train_generator)
val_steps = len(validation_generator)

try:
    # Train the model with better training parameters
    history = model.fit(
        train_generator,
        epochs=50,
        steps_per_epoch=steps_per_epoch,
        validation_data=validation_generator,
        validation_steps=val_steps,
        callbacks=[early_stopping, reduce_lr]
    )

except KeyboardInterrupt:
    pass


Epoch 1/50
100/100 [==============================] - 13s 118ms/step - loss: 1.5359 - accuracy: 0.3395 - val_loss: 1.2635 - val_accuracy: 0.6457 - lr: 0.0010
Epoch 2/50
100/100 [==============================] - 11s 115ms/step - loss: 1.4276 - accuracy: 0.4242 - val_loss: 1.1915 - val_accuracy: 0.6672 - lr: 0.0010
Epoch 3/50
100/100 [==============================] - 12s 120ms/step - loss: 1.3962 - accuracy: 0.4572 - val_loss: 1.0764 - val_accuracy: 0.6511 - lr: 0.0010
Epoch 4/50
100/100 [==============================] - 12s 115ms/step - loss: 1.3302 - accuracy: 0.4966 - val_loss: 1.0959 - val_accuracy: 0.6562 - lr: 0.0010
Epoch 5/50
100/100 [==============================] - 11s 114ms/step - loss: 1.3078 - accuracy: 0.5048 - val_loss: 1.0368 - val_accuracy: 0.6661 - lr: 0.0010
Epoch 6/50
100/100 [==============================] - 11s 115ms/step - loss: 1.2912 - accuracy: 0.5109 - val_loss: 1.0576 - val_accuracy: 0.6468 - lr: 0.0010
Epoch 7/50
100/100 [==============================] 

In [73]:
# Save the model to disk
model.save('model.h5')

In [2]:
from keras.models import load_model

model = load_model('model.h5')

In [8]:
val_steps=validation_generator.n//validation_generator.batch_size+1
loss, acc = model.evaluate_generator(validation_generator,steps=val_steps)
print('Test loss: %f' %loss)
print('Test accuracy: %f' %acc)

C:\Users\andri\AppData\Local\Temp\ipykernel_13456\989374102.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  loss, acc = model.evaluate_generator(validation_generator,steps=val_steps)


Test loss: 0.950330
Test accuracy: 0.669698


In [43]:

from sklearn.metrics import classification_report
import numpy as np
 
preds = model.predict(validation_generator,steps=val_steps)

Ypred = np.argmax(preds, axis=1)
Ytest = validation_generator.classes  # shuffle=False in test_generator

print(classification_report(Ytest, Ypred, labels=None, target_names=classnames, digits=3))

18/18 [==============================] - 2s 99ms/step
              precision    recall  f1-score   support

           0      0.500     0.075     0.131        53
           1      0.287     0.509     0.367       110
           2      0.417     0.512     0.460       162
           3      0.810     0.736     0.771       758
           4      0.000     0.000     0.000        15

    accuracy                          0.638      1098
   macro avg      0.403     0.367     0.346      1098
weighted avg      0.674     0.638     0.643      1098



In [44]:
from sklearn.metrics import confusion_matrix

preds = model.predict(validation_generator,verbose=1,steps=val_steps)

Ypred = np.argmax(preds, axis=1)
Ytest = validation_generator.classes  # shuffle=False in test_generator

cm = confusion_matrix(Ytest, Ypred)

conf = [] # data structure for confusions: list of (i,j,cm[i][j])
for i in range(0,cm.shape[0]):
  for j in range(0,cm.shape[1]):
    if (i!=j and cm[i][j]>0):
      conf.append([i,j,cm[i][j]])

col=2
conf = np.array(conf)
conf = conf[np.argsort(-conf[:,col])]  # decreasing order by 3-rd column (i.e., cm[i][j])

print('%-16s     %-16s  \t%s \t%s ' %('True','Predicted','errors','err %'))
print('------------------------------------------------------------------')
for k in conf:
  print('%-16s ->  %-16s  \t%d \t%.2f %% ' %(classnames[k[0]],classnames[k[1]],k[2],k[2]*100.0/validation_generator.n))
  

18/18 [==============================] - 2s 99ms/step
True                 Predicted         	errors 	err % 
------------------------------------------------------------------
3                ->  1                 	107 	9.74 % 
3                ->  2                 	94 	8.56 % 
2                ->  3                 	58 	5.28 % 
1                ->  3                 	36 	3.28 % 
0                ->  3                 	27 	2.46 % 
0                ->  1                 	15 	1.37 % 
2                ->  1                 	15 	1.37 % 
1                ->  2                 	13 	1.18 % 
4                ->  3                 	9 	0.82 % 
0                ->  2                 	5 	0.46 % 
2                ->  4                 	3 	0.27 % 
4                ->  1                 	3 	0.27 % 
4                ->  2                 	2 	0.18 % 
3                ->  4                 	2 	0.18 % 
2                ->  0                 	1 	0.09 % 
1                ->  0                 	1 	0.09 % 

In [6]:
import sys
import numpy as np
from keras.models import load_model

try:
    import gymnasium as gym
except ModuleNotFoundError:
    print('gymnasium module not found. Try to install with')
    print('pip install gymnasium[box2d]')
    sys.exit(1)

from gymnasium.wrappers import RecordVideo   # Import the Monitor wrapper

model = load_model('model.h5')

def play(env, model):

    seed = 2000
    obs, _ = env.reset(seed=seed)
    
    # drop initial frames
    action0 = 0
    for i in range(50):
        obs,_,_,_,_ = env.step(action0)
    
    done = False
    while not done:
        p = model(np.expand_dims(obs, axis=0)) # reshape input data to have a batch dimension of size 1
        action = np.argmax(p)  # adapt to your model
        obs, _, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

    env.close()

env_arguments = {
    'domain_randomize': False,
    'continuous': False,
    'render_mode': 'rgb_array'
}

env_name = 'CarRacing-v2'
env = gym.make(env_name, **env_arguments)

# Wrap the environment with the Monitor wrapper to record videos
video_dir = 'video_recordings'  # Specify the directory to save video recordings
env = RecordVideo (env, video_dir)

print("Environment:", env_name)
print("Action space:", env.action_space)
print("Observation space:", env.observation_space)

play(env, model)


Environment: CarRacing-v2
Action space: Discrete(5)
Observation space: Box(0, 255, (96, 96, 3), uint8)
Moviepy - Building video c:\Users\andri\Documents\GitHub\Homework-2\video_recordings\rl-video-episode-0.mp4.
Moviepy - Writing video c:\Users\andri\Documents\GitHub\Homework-2\video_recordings\rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready c:\Users\andri\Documents\GitHub\Homework-2\video_recordings\rl-video-episode-0.mp4


## Second Iteration ##

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, BatchNormalization, Dropout, GlobalAveragePooling2D, Activation
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

def lr_scheduler(epoch, lr):
    if epoch % 10 == 0 and epoch != 0:
        lr = lr * 0.9  
    return lr

def AdvancedCNN(num_classes, input_shape):
    model = Sequential()

    # C1 Convolutional Layer 
    model.add(Conv2D(filters=15, input_shape=input_shape, kernel_size=(5,5)))
    model.add(Activation('relu'))

    # C2 Convolutional Layer
    model.add(Conv2D(filters=20, kernel_size=(5,5)))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2)))

    # C3 Convolutional Layer
    model.add(Conv2D(filters=30, kernel_size=(3,3)))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2)))

    # Flatten
    model.add(Flatten())

    # D1 Dense Layer
    model.add(Dense(128))
    model.add(Activation('relu'))
    # Dropout
    model.add(Dropout(0.4))
    # D2 Dense Layer
    model.add(Dense(96))
    model.add(Activation('relu'))
    # Dropout
    model.add(Dropout(0.4))

    # Output Layer
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # Compile the model
    optimizer = Adam(learning_rate=0.001)  # Adjust learning rate as needed
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Print the model summary
    model.summary()

    return model

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest'
)

# Assuming you have 'train' and 'validation' directories for training and validation data
train_generator = train_datagen.flow_from_directory(
    directory='train',
    target_size=(96, 96),
    color_mode='rgb',
    batch_size=64,
    class_mode='categorical',
    shuffle=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    directory='test',
    target_size=(96, 96),
    batch_size=64,
    class_mode='categorical',
    shuffle=False
)

num_classes = train_generator.num_classes
input_shape = train_generator.image_shape

# Train the model using the generators with learning rate scheduler
model = AdvancedCNN(num_classes, input_shape)
lr_schedule = LearningRateScheduler(lr_scheduler)
history = model.fit(train_generator, epochs=100, validation_data=validation_generator, callbacks=[lr_schedule])

Found 6369 images belonging to 5 classes.
Found 2749 images belonging to 5 classes.
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 92, 92, 15)        1140      
                                                                 
 activation_18 (Activation)  (None, 92, 92, 15)        0         
                                                                 
 conv2d_10 (Conv2D)          (None, 88, 88, 20)        7520      
                                                                 
 activation_19 (Activation)  (None, 88, 88, 20)        0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 44, 44, 20)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 

In [10]:
# Save second model
model.save('model2.h5')

In [9]:
import sys
import numpy as np
from keras.models import load_model
from sklearn.metrics import classification_report
import numpy as np

try:
    import gymnasium as gym
except ModuleNotFoundError:
    print('gymnasium module not found. Try to install with')
    print('pip install gymnasium[box2d]')
    sys.exit(1)

model = load_model('model2.h5')

# Classification report
 
preds = model.predict(validation_generator,steps=val_steps)

Ypred = np.argmax(preds, axis=1)
Ytest = validation_generator.classes  # shuffle=False in test_generator

print(classification_report(Ytest, Ypred, labels=None, target_names=classnames, digits=3))


43/43 [==============================] - 2s 36ms/step
              precision    recall  f1-score   support

           0      0.400     0.241     0.300       133
           1      0.361     0.604     0.452       275
           2      0.484     0.724     0.580       406
           3      0.846     0.711     0.773      1896
           4      0.000     0.000     0.000        39

    accuracy                          0.670      2749
   macro avg      0.418     0.456     0.421      2749
weighted avg      0.711     0.670     0.679      2749



In [5]:
import sys
import numpy as np
from keras.models import load_model

try:
    import gymnasium as gym
except ModuleNotFoundError:
    print('gymnasium module not found. Try to install with')
    print('pip install gymnasium[box2d]')
    sys.exit(1)

from gymnasium.wrappers import RecordVideo   # Import the Monitor wrapper

model = load_model('model2.h5')

def play(env, model):

    seed = 2000
    obs, _ = env.reset(seed=seed)
    
    # drop initial frames
    action0 = 0
    for i in range(50):
        obs,_,_,_,_ = env.step(action0)
    
    done = False
    while not done:
        p = model(np.expand_dims(obs, axis=0)) # reshape input data to have a batch dimension of size 1
        action = np.argmax(p)  # adapt to your model
        obs, _, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

    env.close()

env_arguments = {
    'domain_randomize': False,
    'continuous': False,
    'render_mode': 'rgb_array'
}

env_name = 'CarRacing-v2'
env = gym.make(env_name, **env_arguments)

# Wrap the environment with the Monitor wrapper to record videos
video_dir = 'video_recordings'  # Specify the directory to save video recordings
env = RecordVideo (env, video_dir)

print("Environment:", env_name)
print("Action space:", env.action_space)
print("Observation space:", env.observation_space)

play(env, model)

c:\Users\andri\miniconda3\envs\tf\lib\site-packages\gymnasium\wrappers\record_video.py:94: UserWarning: WARN: Overwriting existing videos at c:\Users\andri\Documents\GitHub\Homework-2\video_recordings folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Environment: CarRacing-v2
Action space: Discrete(5)
Observation space: Box(0, 255, (96, 96, 3), uint8)
Moviepy - Building video c:\Users\andri\Documents\GitHub\Homework-2\video_recordings\rl-video-episode-0.mp4.
Moviepy - Writing video c:\Users\andri\Documents\GitHub\Homework-2\video_recordings\rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready c:\Users\andri\Documents\GitHub\Homework-2\video_recordings\rl-video-episode-0.mp4
